<h3 align="center">Codebasics ML Course: Grid Search CV</h3>

We will generate a synthetic dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_features=10, 
    n_samples=1000, 
    n_informative=8,
    n_redundant=2,
    n_repeated=0,
    n_classes=2, 
    random_state=42
)

### Method 1: Evaluate the model using train, test split and tune parameters by trial and error

In [2]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = DecisionTreeClassifier(criterion="entropy", max_depth=10) # criteria: "gini" or "entropy", max_depth=5 or 10
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.84      0.75      0.79       130
           1       0.76      0.84      0.80       120

    accuracy                           0.80       250
   macro avg       0.80      0.80      0.80       250
weighted avg       0.80      0.80      0.80       250



### Method 2: Use K Fold Cross Validation

In [3]:
from sklearn.model_selection import cross_val_score

cross_val_score(DecisionTreeClassifier(criterion="gini", max_depth=5), X, y, cv=5)

array([0.78 , 0.8  , 0.74 , 0.795, 0.77 ])

In [4]:
cross_val_score(DecisionTreeClassifier(criterion="entropy", max_depth=5), X, y, cv=5)

array([0.765, 0.785, 0.765, 0.815, 0.78 ])

In [5]:
criterion = ["gini", "entropy"]
max_depth = [5, 10, 15]

avg_scores = {}

for c in criterion:
    for d in max_depth:
        clf = DecisionTreeClassifier(criterion=c, max_depth=d)
        score_list = cross_val_score(clf, X, y, cv=5)
        avg_scores[c + "_" + str(d)] = np.average(score_list)
        
avg_scores

{'gini_5': 0.775,
 'gini_10': 0.7869999999999999,
 'gini_15': 0.796,
 'entropy_5': 0.781,
 'entropy_10': 0.791,
 'entropy_15': 0.8039999999999999}

### Method 3: Use GridSearchCV

In [6]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(
    DecisionTreeClassifier(),
    {'criterion': ["gini", "entropy"],'max_depth': [5, 10, 15]},
    cv=5,
    return_train_score=False
)
clf.fit(X, y)
clf.cv_results_

{'mean_fit_time': array([0.01852908, 0.02593751, 0.02493157, 0.02436194, 0.02983603,
        0.03473425]),
 'std_fit_time': array([0.00250937, 0.00209429, 0.00183077, 0.0016723 , 0.00512584,
        0.00682766]),
 'mean_score_time': array([0.00239458, 0.00200429, 0.00239663, 0.00197358, 0.00486722,
        0.00322394]),
 'std_score_time': array([8.04843785e-04, 5.31325847e-06, 4.85692983e-04, 5.72152004e-04,
        5.48116753e-03, 6.44788742e-03]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                    'entropy'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[5, 10, 15, 5, 10, 15],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'params': [{'criterion': 'gini', 'max_depth': 5},
  {'criterion': 'gini', 'max_depth': 10},
  {'criterion': 'gini', 'max_depth': 15},
  {'criterion': 'entropy',

In [7]:
df = pd.DataFrame(clf.cv_results_)
df.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018529,0.002509,0.002395,0.000805,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.775,0.815,0.765,0.805,0.770,0.786,0.020100,4
1,0.025938,0.002094,0.002004,0.000005,gini,10,"{'criterion': 'gini', 'max_depth': 10}",0.785,0.710,0.785,0.780,0.815,0.775,0.034785,6
2,0.024932,0.001831,0.002397,0.000486,gini,15,"{'criterion': 'gini', 'max_depth': 15}",0.795,0.730,0.835,0.790,0.815,0.793,0.035299,2


In [8]:
df[["param_criterion", "param_max_depth", "mean_test_score"]]

,param_criterion,param_max_depth,mean_test_score
0,gini,5,0.786
1,gini,10,0.775
2,gini,15,0.793
3,entropy,5,0.778
4,entropy,10,0.792
5,entropy,15,0.804


In [9]:
clf.best_params_

{'criterion': 'entropy', 'max_depth': 15}

In [10]:
model = clf.best_estimator_
model

DecisionTreeClassifier(criterion='entropy', max_depth=15)

### Now let's try different models with different parameters

In [11]:
from sklearn import svm

model_params = {
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params' : {
            'criterion': ["gini", "entropy"],
            'max_depth': [5, 10, 15]
        }  
    },
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']            
        }
    }
}

scores = []

for key, val in model_params.items():
    clf = GridSearchCV(
        val['model'],
        val['params'],
        cv=5,
        return_train_score=False
    )
    clf.fit(X, y)
    scores.append({
        'model': key,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
scores

[{'model': 'decision_tree',
  'best_score': 0.817,
  'best_params': {'criterion': 'entropy', 'max_depth': 15}},
 {'model': 'svm',
  'best_score': 0.9260000000000002,
  'best_params': {'C': 1, 'kernel': 'rbf'}}]

In [12]:
df = pd.DataFrame(scores, columns=["model", "best_score", "best_params"])
df

,model,best_score,best_params
0,decision_tree,0.817,"{'criterion': 'entropy', 'max_depth': 15}"
1,svm,0.926,"{'C': 1, 'kernel': 'rbf'}"
